In [1]:
import json
from pprint import pprint
import glob
import os
import re
import cv2
import pandas as pd
import numpy as np

In [2]:
class Points():
    def __init__(self):
        self.frameNumber = -1
        self.xLeft = []
        self.xRight = []
        self.yLeft = []
        self.yRight = []
        self.cLeft = []
        self.cRight = []
        self.xBody = []
        self.yBody = []
        self.cBody = []
        self.totalCLeft = 0
        self.totalCRight = 0
    

In [3]:
class Distance():
    def __init__(self):
        self.left = []
        self.right = []
        self.avg = 0
        self.rank = -100
        self.idxWord = -1
        self.idxTrans = -1

In [4]:
class SortedCutPoint:
    def __init__(self):
        self.idxDistTransA = -1
        self.idxDistTransB = -1
        self.totalDifIdx = -1
        self.frameIdx = -1

In [5]:
def LoadTransition():
    trans = []
#     point_ins = Points()
#     point_ins.xLeft = []
    files = glob.glob("/home/yulia/openpose/outputScene8-1/*.json")
    ctrTrans = 0
    for f in files:
        fName = os.path.basename(f)
        number = re.findall('\d+',f)
        number = max(map(int,number))
        trans.append(Points())
        with open(f) as df:
            data = json.load(df)
            frameNumber = number
            trans[ctrTrans].frameNumber = number
            #each hand has 21 points. x,y,c --> 0 to 62
            for i in range(0,21):
                trans[ctrTrans].xLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3])
                trans[ctrTrans].yLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+1])
                trans[ctrTrans].cLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+2])
                trans[ctrTrans].xRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3])
                trans[ctrTrans].yRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+1])
                trans[ctrTrans].cRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+2])
                trans[ctrTrans].totalCLeft += data['people'][0]['hand_left_keypoints_2d'][i*3+2]
                trans[ctrTrans].totalCRight += data['people'][0]['hand_right_keypoints_2d'][i*3+2]
            for i in range(0, 25):
                trans[ctrTrans].xBody.append(data['people'][0]['pose_keypoints_2d'][i*3])
                trans[ctrTrans].yBody.append(data['people'][0]['pose_keypoints_2d'][i*3+1])
                trans[ctrTrans].cBody.append(data['people'][0]['pose_keypoints_2d'][i*3+2])
        ctrTrans+=1
    return trans

In [6]:
trans = LoadTransition()

In [7]:
len(trans)

1332

In [8]:
wordsFrame = []
wordsFrame.append(Points())
ctrWordsFrame = 0
with open("/home/yulia/openpose/african_american/african_american_000000000060_keypoints.json") as df:
    data = json.load(df)
    wordsFrame[ctrWordsFrame].frameNumber = ctrWordsFrame
    #each hand has 21 points. x,y,c --> 0 to 62
    for i in range(0,21):
        wordsFrame[ctrWordsFrame].xLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+2])
        wordsFrame[ctrWordsFrame].xRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+2])
    for i in range(0, 25):
        wordsFrame[ctrWordsFrame].xBody.append(data['people'][0]['pose_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yBody.append(data['people'][0]['pose_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cBody.append(data['people'][0]['pose_keypoints_2d'][i*3+2])

ctrWordsFrame+=1
wordsFrame.append(Points())
with open("/home/yulia/openpose/advisor1/advisor1_000000000000_keypoints.json") as df:
    data = json.load(df)
    wordsFrame[ctrWordsFrame].frameNumber = ctrWordsFrame

    #each hand has 21 points. x,y,c --> 0 to 62
    for i in range(0,21):
        wordsFrame[ctrWordsFrame].xLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+2])
        wordsFrame[ctrWordsFrame].xRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+2])
    for i in range(0, 25):
        wordsFrame[ctrWordsFrame].xBody.append(data['people'][0]['pose_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yBody.append(data['people'][0]['pose_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cBody.append(data['people'][0]['pose_keypoints_2d'][i*3+2])

In [9]:
transFrame = []
dist = []
transFrame.append(wordsFrame[0])
transFrame.append(wordsFrame[1])
for loop in range(0,3):
    for i in range(0,len(transFrame)):
        if(len(dist) <= i or dist[i][0].idxWord != transFrame[i].frameNumber):
            dist.insert(i,[])
#             print(str(i) + " || " + str(len(transFrame)))
            for j in range(0, len(trans)):
                dist[i].append(Distance())
                for k in range(0, 21):
                    x1 = transFrame[i].xLeft[k]
                    x2 = trans[j].xLeft[k]
                    y1 = transFrame[i].yLeft[k]
                    y2 = trans[j].yLeft[k]
                    dist[i][j].left.append(np.sqrt(np.power(x1-x2,2)+np.power(y1-y2,2)))
                    x1 = transFrame[i].xRight[k]
                    x2 = trans[j].xRight[k]
                    y1 = transFrame[i].yLeft[k]
                    y2 = trans[j].yLeft[k]
                    dist[i][j].right.append(np.sqrt(np.power(x1-x2,2)+np.power(y1-y2,2)))
                    dist[i][j].avg = np.mean(dist[i][j].left + dist[i][j].right)
#                     dist[i][j].avg = np.max(dist[i][j].left + dist[i][j].right)
                    dist[i][j].idxWord = transFrame[i].frameNumber
                    dist[i][j].idxTrans = trans[j].frameNumber
    for i in range(0, len(transFrame)):
        dist[i].sort(key = lambda x: x.avg, reverse = False)
#         print("=======================================================")
#         for j in range(0, len(trans)):
#             print(str(dist[i][j].avg)  + " : " + str(dist[i][j].idxTrans) )
    
    frameNum = len(transFrame)
    tempTransFrame = []
    for n in range(0, frameNum-1):
        tempTransFrame.append(transFrame[n])
        sortedCutPoint = []
        for i in range(0, len(trans)):
            sortedCutPoint.append(SortedCutPoint())
            for j in range(0, len(trans)):
                if dist[n][i].idxTrans == dist[n+1][j].idxTrans:
                    sortedCutPoint[i].idxDistTransA = i
                    sortedCutPoint[i].idxDistTransB = j
                    sortedCutPoint[i].totalDifIdx = i+j
                    sortedCutPoint[i].frameIdx = dist[n][i].idxTrans
                    break
        sortedCutPoint.sort(key = lambda x: x.totalDifIdx, reverse = False)
#         for i in range(0, len(trans)):
#             print(sortedCutPoint[i].idxDistTransA)
#             print(sortedCutPoint[i].idxDistTransB)
#             print(sortedCutPoint[i].totalDifIdx)
        print(sortedCutPoint[0].frameIdx)
        print("===================================")
        fn = sortedCutPoint[0]
        temp_list = list(filter(lambda x: x.frameNumber == sortedCutPoint[0].frameIdx, trans))
        trans = list(filter(lambda x : x.frameNumber != sortedCutPoint[0].frameIdx, trans))
        for i in range(0,len(dist)):
            dist[i] = list(filter(lambda x : x.idxTrans != sortedCutPoint[0].frameIdx, dist[i]))
#         transFrame.insert(n+1,temp_list[0])
        tempTransFrame.append(temp_list[0])
    tempTransFrame.append(transFrame[len(transFrame)-1])
    transFrame = tempTransFrame.copy()
    del(tempTransFrame)

942
1243
245
941
246
943
282


In [10]:
def readVideo(vid):
    cap = vid
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(frameWidth)
    
    buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
    
    fc = 0
    ret = True

    while (fc < frameCount  and ret):
        ret, buf[fc] = cap.read()
        fc += 1

    cap.release()
    return buf

In [11]:
#read transition video and save to array of images
vidTrans = cv2.VideoCapture('/home/yulia/Documents/scene8-camera1.mp4')
vidAfricanAmerican = cv2.VideoCapture('/home/yulia/Documents/ASL/african_american.mp4')
vidAdvisor = cv2.VideoCapture('/home/yulia/Documents/ASL/advisor1.mp4')
transImg = readVideo(vidTrans)

1920


In [12]:
wordsVid = []
wordsVid.append(readVideo(vidAfricanAmerican))
wordsVid.append(readVideo(vidAdvisor))

1920
1920


In [13]:
writtenTrans = []
transFrame.pop(0)
transFrame.pop(len(transFrame)-1)
for i in range(0,len(transFrame)):
    writtenTrans.append(transImg[transFrame[i].frameNumber])
# for i in range(0,sortedCutPoint[0].idxDistTransA):
#     writtenTrans.append(transImg[dist[0][i].idxTrans])
# for i in range(sortedCutPoint[0].idxDistTransB-1, -1, -1):
#     writtenTrans.append(transImg[dist[1][i].idxTrans])

In [14]:
for i in range(0,len(transFrame)):
    print(transFrame[i].frameNumber)

941
1243
246
942
943
245
282


In [15]:
totalWrittenFrames = []
totalWrittenFrames.append(wordsVid[0])
totalWrittenFrames.append(writtenTrans)
totalWrittenFrames.append(wordsVid[1])

In [16]:
height,width,channels = wordsVid[0][0].shape
out = cv2.VideoWriter('/home/yulia/outpy3_10.mp4',cv2.VideoWriter_fourcc('F','M','P', '4'), 10, (width,height))
# out = cv2.VideoWriter('/home/yulia/outpyMax2.mp4',cv2.VideoWriter_fourcc('F','M','P', '4'), 10, (width,height))
# out.set(cv2.CAP_PROP_FPS,300)
# out.set(cv2.CAP_PROP_FPS, vidTrans.get(cv2.CAP_PROP_FPS))
for i in range(0, len(totalWrittenFrames)):
    for frame in range(0, len(totalWrittenFrames[i])):
        out.write(totalWrittenFrames[i][frame])
        print(frame)
out.release()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [17]:
len(totalWrittenFrames[1])

7